In [ ]:
#First install the necesary libraries
!pip install openai==1.1.1
!pip install panel

In [ ]:
import openai  # OpenAI for GPT models
import geopy  # Geocoding services
import panel as pn  # Interactive apps
openai.api_key="your-api-key"  # API key for OpenAI

In [ ]:
def continue_conversation(messages, temperature=0.7):
    # Generates a chat completion using GPT-4 with given messages and temperature
    response = openai.chat.completions.create(
        model="gpt-4-0613",
        messages=messages,
        temperature=temperature,
    )
    answer = response.choices[0].message.content  # Extracts the response content
    
    return answer

In [ ]:
import panel as pn
pn.extension()  # Load Panel extensions for interactive widgets
panels = []  # Initialize an empty list to store panels
client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')  # Text input widget for user prompts
button_conversation = pn.widgets.Button(name="talk")  # Button widget to trigger conversation


In [ ]:
system_prompt = f"""
You are a discovery chatbot. Your goal is to assist users with by finding various options as per need and assist in bookings.
First you try to find intent of user. Once user intent is found, identify entities required for search and fill those slots.
If user doesn't provide them, ask it to user one by one.
We only help user for specific intents only.
Once you have all the details, write a sql query assuming a simple booking, and sort them by the factor as specified. Make sure that in sql query, you use same names as entities provided below for the intent.

Intent: Restraunt booking; Entities:[cuisine, budget, number of tables]; Sort by: rating
Intent: Flight ticket booking; Entities: [from, to, date]; Sort by: rating

"""
context = [{'role': 'system', 'content': system_prompt}]  # System message defines chatbot role, intents, and entity requirements for bookings

In [ ]:
def add_prompts_conversation(_):
    # Get user input from text field and reset input field
    prompt = client_prompt.value_input
    client_prompt.value = ''
    
    # Add user's input to the conversation context
    context.append({'role': 'user', 'content': f"{prompt}"})
    
    # Generate response from the assistant
    response = continue_conversation(context)
    
    # Append assistant's response to the conversation context
    context.append({'role': 'assistant', 'content': f"{response}"})
    
    # Display user and assistant messages in the panel
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
    
    return pn.Column(*panels)  # Return updated conversation in a column layout


In [ ]:
interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)  # Binds the button to trigger conversation updates

dashboard = pn.Column(
    client_prompt,  # Text input for user prompts
    pn.Row(button_conversation),  # Button to initiate conversation
    pn.panel(interactive_conversation, loading_indicator=True),  # Displays the conversation with a loading indicator
)

dashboard  # Render the dashboard with input, button, and conversation display